## Rerun of acquiring DOIs using the Crossref API with the python library 'habanero' 
(conda installed v0.6)
check with first ten 



code for basic install and a simple search of the works using a query and a filter



In [1]:
import pandas as pd
from habanero import Crossref, exceptions
import time

### First bring in the existing list of publictaions (using the columns needed)

In [2]:
df1 = pd.read_csv('./A3out_CrossRef_BRC_FuzzyMatched_Oct19.csv', index_col=0)
df1.shape

(2378, 18)

In [ ]:
df1.head(2)

In [ ]:
#make dataframe from only those missing DOIs
#df1[~df1.doi.isna()]  gives those with DOIs (1440 as reported)
df1[df1.good_list==False].shape


### Therefore identifying that 620 of the 2378 publications without  DOI matches
### _so it will best to acquire all top matches from CrossRef using the full 'complete' reference from the original file_


In [3]:
cr = Crossref()

In [4]:
#Crossref(api_key = "168421")
# set a mailto address
Crossref(mailto = "your e-mail here")

< Crossref 
URL: https://api.crossref.org
KEY: None
MAILTO: <your e-mail here
ADDITIONAL UA STRING: None
>

In [15]:
crossref_test =cr.works(query=df1.complete[26]
             ,select=['title','DOI'],
                      filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)

In [16]:
crossref_test['message']['items'][0]['DOI'],crossref_test['message']['items'][0]['title'][0]

('10.3389/fmicb.2015.01016',
 'Ongoing burden of disease and mortality from HIV/CMV coinfection in Africa in the antiretroviral therapy era')

In [20]:
def CrossRefDoiRetry(df):
    try:
        search = cr.works(query=df.complete
             ,select=['title','DOI'],filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)
        time.sleep(1)
        return search['message']['items'][0]['DOI'], search['message']['items'][0]['title'][0]
    except:
        return None
    finally:
        time.sleep(1)
        pass

In [21]:
df_retry =df1[df1.good_list==False]

In [22]:
df_retry[['DOI_retry_API','title_retry_API']]= df_retry.apply(CrossRefDoiRetry, axis=1, result_type='expand')

C:\Users\labrown\Miniconda3\envs\biblio-edge\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [27]:
df_retry.columns

Index(['ID', 'DOI', 'complete', 'csv_post_title', 'crossref_API_out', 'title',
       'DOI_API', 'title_API', 'title_match', 'DOI_match', 'CrossRefTitle_low',
       'DOI_fuzzyRatio', 'Title_fuzzyRatio', 'Title_partialRatio',
       'Complete_fuzzyRatio', 'Complete_partialRatio', 'TitleMatch01',
       'good_list', 'DOI_retry_API', 'title_retry_API'],
      dtype='object')

In [45]:
df1_out = df1.merge(df_retry, how='left')

In [46]:
df1_out.to_csv('./A4out_CrossRef_FuzzyMatched_with_retry_Oct19.csv')

In [47]:
df1_out

,ID,DOI,complete,csv_post_title,crossref_API_out,title,DOI_API,title_API,title_match,DOI_match,...,DOI_fuzzyRatio,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,TitleMatch01,good_list,good_list2,DOI_retry_API,title_retry_API
0,1125,10.1186/s12881-014-0095-4,"&amp; , fenwick al, goos jac, rankin j, lord h...",apparently synonymous substitutions in fgfr2 a...,{'title': ['Apparently synonymous substitution...,['Apparently synonymous substitutions in FGFR2...,10.1186/s12881-014-0095-4,Apparently synonymous substitutions in FGFR2af...,False,True,...,100.0,99.0,99.0,51.0,99.0,0,True,True,NaN,NaN
1,1996,10.1183/13993003.00321-2016,", pattinson kt, turner mr. a wider pathologica...",a wider pathological network underlying breath...,{'title': ['A wider pathological network under...,['A wider pathological network underlying brea...,10.1183/13993003.00321-2016,A wider pathological network underlying breath...,True,True,...,100.0,100.0,100.0,73.0,100.0,1,True,True,NaN,NaN
2,506,NaN,"adib-samii p, rost n, traylor m, devan w, ...",17q25 locus is associated with white matter hy...,{'title': ['17q25 Locus Is Associated With Whi...,['17q25 Locus Is Associated With White Matter ...,10.1161/strokeaha.113.679936,17q25 Locus Is Associated With White Matter Hy...,True,False,...,NaN,100.0,100.0,25.0,100.0,1,True,True,NaN,NaN
3,1430,10.1093/annonc/mdu449,"and i. tomlinson*, findlay jm, middleton mr, t...",a systematic review and meta-analysis of somat...,{'title': ['A systematic review and meta-analy...,['A systematic review and meta-analysis of som...,10.1093/annonc/mdu449,A systematic review and meta-analysis of somat...,True,True,...,100.0,100.0,100.0,72.0,100.0,1,True,True,NaN,NaN
4,848,NaN,"dichgans m, malik r, kã•_nig ir, rosand j, ...",shared genetic susceptibility to ischemic stro...,{'title': ['Shared Genetic Susceptibility to I...,['Shared Genetic Susceptibility to Ischemic St...,10.1161/strokeaha.113.002707,Shared Genetic Susceptibility to Ischemic Stro...,False,False,...,NaN,78.0,100.0,14.0,100.0,0,True,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,1838,10.1007/s00109-015-1377-9,"zois ce, harris al. glycogen metabolism has a ...",glycogen metabolism has a key role in the canc...,{'title': ['Glycogen metabolism has a key role...,['Glycogen metabolism has a key role in the ca...,10.1007/s00109-015-1377-9,Glycogen metabolism has a key role in the canc...,True,True,...,100.0,100.0,100.0,70.0,100.0,1,True,True,NaN,NaN
2374,1103,10.1093/brain/awu143,"zokaei n, mcneill a, proukakis c, beavan m, ja...",visual short-term memory deficits associated w...,{'title': ['Visual short-term memory deficits ...,['Visual short-term memory deficits associated...,10.1093/brain/awu143,Visual short-term memory deficits associated w...,False,True,...,100.0,99.0,99.0,47.0,98.0,0,True,True,NaN,NaN
2375,1065,10.1212/NXI.0000000000000016,"zuliani l, ferlazzo e, andrigo c, casano a, ci...","glycine receptor antibodies in 2 cases of new,...",{'title': ['Glycine receptor antibodies in 2 c...,['Glycine receptor antibodies in 2 cases of ne...,10.1212/nxi.0000000000000016,"Glycine receptor antibodies in 2 cases of new,...",False,True,...,89.0,99.0,100.0,37.0,100.0,0,True,True,NaN,NaN
2376,69,NaN,"zuliani l, graus f, giometto b, bien c, vincen...",central nervous system neuronal surface antibo...,{'title': ['Central nervous system neuronal su...,['Central nervous system neuronal surface anti...,10.1136/jnnp-2011-301237,Central nervous system neuronal surface antibo...,True,False,...,NaN,100.0,100.0,75.0,100.0,1,True,True,NaN,NaN


## How many references remain unmatched?

> we can create anyother check to see if the new title and DOI match the records we started with, then see how many remain missing from the lists of matched references. 

In [48]:
df_matched2 = df1_out.assign(
    title_match2 = df1_out.csv_post_title.str.strip().str.lower()==df1_out.title_retry_API.str.strip().str.lower(),
    DOI_match2 = df1_out.DOI.str.strip().str.lower()==df1_out.DOI_retry_API.str.strip().str.lower() )

In [49]:
df_matched2['good_list2'] = ((df_matched2.DOI_match==True)|(df_matched2.DOI_match2==True)|
                     (df_matched2.title_match==True)|(df_matched2.title_match2==True))
                     

In [53]:
df_matched2.groupby(['good_list','good_list2']).count()

ID   DOI  complete  csv_post_title  crossref_API_out  \
good_list good_list2                                                           
False     False        176    50       176             176               176   
          True         444   316       444             444               444   
True      False        222    39       222             222               222   
          True        1536  1037      1536            1536              1536   

                      title  DOI_API  title_API  title_match  DOI_match  ...  \
good_list good_list2                                                     ...   
False     False         176      176         80          176        176  ...   
          True          444      444        244          444        444  ...   
True      False         222      222        222          222        222  ...   
          True         1536     1536       1536         1536       1536  ...   

                      DOI_fuzzyRatio  Title_fuzzyRatio  Title_partialRatio  \
good_list good_list2                                                         
False     False                   50                80                  80   
          True                   316               244                 244   
True      False                   39               222                 222   
          True                  1037              1536                1536   

                      Complete_fuzzyRatio  Complete_partialRatio  \
good_list good_list2                                               
False     False                        80                     80   
          True                        244                    244   
True      False                       222                    222   
          True                       1536                   1536   

                      TitleMatch01  DOI_retry_API  title_retry_API  \
good_list good_list2                                                 
False     False                176            118              118   
          True                 444            444              444   
True      False                222              0                0   
          True                1536              0                0   

                      title_match2  DOI_match2  
good_list good_list2                            
False     False                176         176  
          True                 444         444  
True      False                222         222  
          True                1536        1536  

[4 rows x 21 columns]

### Note:  The 176 final 'badlist' references, those where matches to tiles or DOIs were no found after checking CrossRef twice, were then manually checked, and this information was then cleaned to return a 'curated DOI' for these references.

##### File names : 
    - Outbound:  A4out_badlist_API_retry(Oct19).csv
    - Working: A5in_curated_API_list(Oct19.xls  and
    - For final DOI list: A5in_curated_API_list(14Oct19).csv.

In [56]:
manual_check =df_matched2[(df_matched2.good_list2==False)&(df_matched2.good_list==False)].reindex(columns=['ID', 'DOI','DOI_API','DOI_retry_API', 'complete',
                          'csv_post_title', 'title_API', 'title_retry_API',
                          'good_list'])

manual_check.to_csv('./A4out_badlist_API_retry(Oct19).csv')